# Импорт необходимых библиотек и модулей / Import necessary libraries and modules

In [46]:
import os
import json
import plotly.graph_objects as go
import random

# Настройка путей к результам / Setting paths to results

In [53]:
current_directory = os.getcwd()
results_folder_name = "Results"
results_directory = os.path.join(current_directory, results_folder_name)
results_file_name = "Results.json"
results_file_path = os.path.join(results_directory, results_file_name)

with open(results_file_path, "r") as json_file:
    loaded_results_data = json.load(json_file)

# Определение ключевых слов для обращения к результатам из JSON файла / Defining keywords for accessing results from a JSON file

In [48]:
resuts_key_fluid_analysis = "Fluid_Analysis" # Ключ для расчетных значений в рамках FA
resuts_key_p_dis_water = "P_dis_water_case" # Ключ для расчетных значений в рамках анализа расчета перепада давления, создаваемым ЭЦН, при прокачке воды
resuts_key_p_dis_oil = "P_dis_oil_case" # Ключ для расчетных значений в рамках анализа расчета перепада давления, создаваемым ЭЦН, при прокачке ГЖС
resuts_key_p_dis_pipesim_dif_gor = "P_dis_pipesim_case_dif_gor"
resuts_key_p_dis_unifloc_dif_gor = "P_dis_unifloc_case_dif_gor"

results_key_pipesim = "PIPESIM" # Ключ, который характерезует данные, как рассчитанные в PIPESIM 
results_key_unifloc = "Unifloc" # Ключ, который характерезует данные, как рассчитанные в Unifloc

# Задаем ключи для параметров
pressure_key_bara = "pressure_bara"
pressure_key_atma = "pressure_atma"
viscosity_key_cp = "viscosity_cP"
temperature_key_C = "temperature_C"
solution_gas_key_m3m3 = "solution_gas_C"
fvf_key = "formation_volume_factor"
density_key_kg_m3 = "denstiy_kg_m3"
liquid_flowrate_key_sm3_day = "liquid_flowrate_sm3_day"

p_dis_oil_key = "Pdis_vs_Q_by_GOR" 

empty_key = "Empty"
empty_value = "Empty"

# Создание функции для сравнения графиков результатов PIPESIM и Unifloc / Creating a function for comparing the results graphs of PIPESIM and Unifloc

In [38]:
def create_comparison_plot(x_data_pipesim, y_data_pipesim, x_data_unifloc, y_data_unifloc, title, x_axis_title, y_axis_title, legend_pipesim, legend_unifloc):
    # Определение графиков / Define the traces
    trace_pipesim = go.Scatter(
        x=x_data_pipesim, 
        y=y_data_pipesim,
        mode='lines+markers', 
        name=legend_pipesim
    )

    trace_unifloc = go.Scatter(
        x=x_data_unifloc, 
        y=y_data_unifloc, 
        mode='lines+markers', 
        name=legend_unifloc
    )

    # Создание макета для графика / Create the layout for the plot
    layout = go.Layout(
        title=title,
        xaxis=dict(title=x_axis_title),
        yaxis=dict(title=y_axis_title)
    )

    # Создание графика / Create the figure
    fig = go.Figure(data=[trace_pipesim, trace_unifloc], layout=layout)

    # Отображение графика / Display the plot
    fig.show()

# Распределение температуры в PIPESIM / Temperature distribution in PIPESIM

In [39]:
if empty_key not in loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim]:
    # Определение данных температуры из модели PIPESIM
    trace_pipesim_temperature = go.Scatter(x=loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara], 
                                        y=loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][temperature_key_C], 
                                        mode='lines+markers', 
                                        name='PIPESIM Temperature')

    # Создание макета графика для температуры
    layout_temperature = go.Layout(title='Pressure vs Temperature', 
                                xaxis=dict(title='Pressure (bara)'), 
                                yaxis=dict(title='Temperature (C)'))


    fig_temperature = go.Figure(data=trace_pipesim_temperature, layout=layout_temperature)
    fig_temperature.show()
else:
    print("Результаты расчета отсутствуют.")

Убеждаемся, что температура, использованная в модели PIPESIM, остается постоянной.

# Сравнение результатов расчета вязкости / Comparison of Viscosity Calculation Results

In [40]:
if empty_key not in loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim]:
    create_comparison_plot(
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara],
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][viscosity_key_cp], 
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara], # (atma) уже переведены в (bara) 
        loaded_results_data[resuts_key_fluid_analysis][results_key_unifloc][viscosity_key_cp],
        'Pressure vs Viscosity Comparison',
        'Pressure (bara)',
        'Viscosity (cP)',
        'PIPESIM Viscosity',
        'Unifloc Viscosity (PVT_mu_oil_cP)'
    )
else:
    print("Результаты расчета отсутствуют.")

# Сравнение результатов расчета газосодержания в PIPESIM и Unifloc VBA / # Comparison of solution gas

In [41]:
if empty_key not in loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim]:
    create_comparison_plot(
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara],
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][solution_gas_key_m3m3], 
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara], # (atma) уже переведены в (bara) 
        loaded_results_data[resuts_key_fluid_analysis][results_key_unifloc][solution_gas_key_m3m3],
        'Solution Gas vs Pressure',
        'Pressure (bara)',
        'Solution Gas (sm3/sm3)',
        'Solution Gas in PIPESIM',
        'Solution Gas in Unifloc'
    )
else:
    print("Результаты расчета отсутствуют.")

# Сравнение результатов расчета Объемного Коэф. в PIPESIM и Unifloc VBA / Comparison of formation volume factor

In [42]:
if empty_key not in loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim]:
    create_comparison_plot(
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara],
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][fvf_key], 
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara], # (atma) уже переведены в (bara) 
        loaded_results_data[resuts_key_fluid_analysis][results_key_unifloc][fvf_key],
        'FVF vs Pressure',
        'Pressure (bara)',
        'FVF',
        'FVF in PIPESIM',
        'FVF in Unifloc'
    )
else:
    print("Результаты расчета отсутствуют.")

# Сравнение результатов расчета вязкости в PIPESIM и Unifloc VBA / Comparison of viscosity

In [43]:
if empty_key not in loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim]:
    create_comparison_plot(
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara],
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][density_key_kg_m3], 
        loaded_results_data[resuts_key_fluid_analysis][results_key_pipesim][pressure_key_bara], # (atma) уже переведены в (bara) 
        loaded_results_data[resuts_key_fluid_analysis][results_key_unifloc][density_key_kg_m3],
        'Density vs Pressure',
        'Pressure (bara)',
        'Density (kg/m3)',
        'Density in PIPESIM',
        'Density in Unifloc'
    )
else:
    print("Результаты расчета отсутствуют.")

# Основные выводы / Main conclusions

- Результаты расчетов вязкости и газосодержания одинаковые
- Есть незначительные расхождения в объемном коэффициенте и вязкости

# Сравнение расчета давления на выходе ЭЦН в PIPESIM и Unifloc (флюид - вода) / Comparison of ESP Discharge Pressure Calculation in PIPESIM and Unifloc (fluid - water)

In [44]:
if empty_key not in loaded_results_data[resuts_key_p_dis_water][results_key_pipesim]:
    # Выбор случайного цвета для построения графика
    random_color = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'

    fig = go.Figure()

    # Добавление линий на график
    fig.add_trace(go.Scatter(x=loaded_results_data[resuts_key_p_dis_water][results_key_pipesim][liquid_flowrate_key_sm3_day], 
                            y=loaded_results_data[resuts_key_p_dis_water][results_key_pipesim][pressure_key_atma],
                            mode='lines', name='PIPESIM', line=dict(color=random_color)))

    fig.add_trace(go.Scatter(x=loaded_results_data[resuts_key_p_dis_water][results_key_unifloc][liquid_flowrate_key_sm3_day], 
                            y=loaded_results_data[resuts_key_p_dis_water][results_key_unifloc][pressure_key_atma],
                            mode='lines', name='Unifloc', line=dict(color=random_color, dash='dash')))

    # Обновление графика
    fig.update_layout(
        title='Pressure Discharge Characteristics of ESP',
        xaxis_title='Liquid Flow Rate (sm3/day)',
        yaxis_title='Pressure (atma)',
        legend_title='Source'
    )

    # Отображение графика
    fig.show()
else:
    print("Результаты расчета отсутствуют.")

Результаты расчета давления на выходе из ЭЦН (флюид - вода) в PIPESIM и Unifloc совпадают.

# Сравнение расчета давления на выходе ЭЦН в PIPESIM и Unifloc (флюид - нефть) / Comparison of ESP Discharge Pressure Calculation in PIPESIM and Unifloc (fluid - oil)

In [45]:
if empty_key not in loaded_results_data[resuts_key_p_dis_oil][results_key_pipesim]:

    fig = go.Figure()

    # Доступные цвета для линий
    all_colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange', 'purple', 'brown']
    used_colors = set()

    # Итерируем по GOR
    for gor_value in loaded_results_data[resuts_key_p_dis_oil][results_key_pipesim][p_dis_oil_key]:
        color = all_colors[len(used_colors) % len(all_colors)]
        used_colors.add(color)

        # Получаем данные для PIPESIM и Unifloc
        pressure_pipesim = loaded_results_data[resuts_key_p_dis_oil][results_key_pipesim][p_dis_oil_key][gor_value][pressure_key_atma]
        pressure_unifloc = loaded_results_data[resuts_key_p_dis_oil][results_key_unifloc][p_dis_oil_key][gor_value][pressure_key_atma]
        liquid_rate_pipesim = loaded_results_data[resuts_key_p_dis_oil][results_key_pipesim][p_dis_oil_key][gor_value][liquid_flowrate_key_sm3_day]
        liquid_rate_unifloc = loaded_results_data[resuts_key_p_dis_oil][results_key_unifloc][p_dis_oil_key][gor_value][liquid_flowrate_key_sm3_day]
        
        # Добавляем данные для PIPESIM
        fig.add_trace(go.Scatter(x=liquid_rate_pipesim , y=pressure_pipesim, mode='lines',
                                name=f'PIPESIM (GOR = {gor_value})', line=dict(color=color)))
        
        # Добавляем данные для Unifloc
        fig.add_trace(go.Scatter(x=liquid_rate_unifloc, y=pressure_unifloc, mode='lines',
                                name=f'Unifloc (GOR = {gor_value})', line=dict(color=color, dash='dash')))

    # Настройки графика
    fig.update_layout(
        title='Dependence of Discharge Pressure on Liquid Flow Rate for Different GOR values',
        xaxis_title='Liquid Flow Rate (sm3/day)',
        yaxis_title='Pressure (atma)',
        legend_title='Source'
    )

    # Показываем график
    fig.show()
else:
    print("Результаты расчета отсутствуют.")

Значения давления на выходе ЭЦН при расчете в PIPESIM и Unifloc отличаются. Чем больше GOR, тем больше отличие.

# Сравнение расчета давления на выходе ЭЦН в PIPESIM (флюид - нефть) / Comparison of ESP Discharge Pressure Calculation in PIPESIM (fluid - oil)

In [52]:
if empty_key not in loaded_results_data[resuts_key_p_dis_pipesim_dif_gor][results_key_pipesim]:

    fig = go.Figure()

    # Доступные цвета для линий
    all_colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange', 'purple', 'brown']
    used_colors = set()

    # Итерируем по GOR
    for gor_value in loaded_results_data[resuts_key_p_dis_pipesim_dif_gor][results_key_pipesim][p_dis_oil_key]:
        color = all_colors[len(used_colors) % len(all_colors)]
        used_colors.add(color)

        # Получаем данные для PIPESIM и Unifloc
        pressure_pipesim = loaded_results_data[resuts_key_p_dis_pipesim_dif_gor][results_key_pipesim][p_dis_oil_key][gor_value][pressure_key_atma]
        liquid_rate_pipesim = loaded_results_data[resuts_key_p_dis_pipesim_dif_gor][results_key_pipesim][p_dis_oil_key][gor_value][liquid_flowrate_key_sm3_day]
        
        # Добавляем данные для PIPESIM
        fig.add_trace(go.Scatter(x=liquid_rate_pipesim , y=pressure_pipesim, mode='lines',
                                name=f'PIPESIM (GOR = {gor_value})', line=dict(color=color)))

    # Настройки графика
    fig.update_layout(
        title='Dependence of Discharge Pressure on Liquid Flow Rate for Different GOR values',
        xaxis_title='Liquid Flow Rate (sm3/day)',
        yaxis_title='Pressure (atma)',
        legend_title='Source'
    )

    # Показываем график
    fig.show()
else:
    print("Результаты расчета отсутствуют.")

# Сравнение расчета давления на выходе ЭЦН в Unifloc (флюид - нефть) / Comparison of ESP Discharge Pressure Calculation in Unifloc (fluid - oil)

In [56]:
if empty_key not in loaded_results_data[resuts_key_p_dis_unifloc_dif_gor][results_key_unifloc]:

    fig = go.Figure()

    # Доступные цвета для линий
    all_colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange', 'purple', 'brown']
    used_colors = set()

    # Итерируем по GOR
    for gor_value in loaded_results_data[resuts_key_p_dis_unifloc_dif_gor][results_key_unifloc][p_dis_oil_key]:
        color = all_colors[len(used_colors) % len(all_colors)]
        used_colors.add(color)

        # Получаем данные для Unifloc
        pressure_unifloc = loaded_results_data[resuts_key_p_dis_unifloc_dif_gor][results_key_unifloc][p_dis_oil_key][gor_value][pressure_key_atma]
        liquid_rate_unifloc = loaded_results_data[resuts_key_p_dis_unifloc_dif_gor][results_key_unifloc][p_dis_oil_key][gor_value][liquid_flowrate_key_sm3_day]
        
        # Добавляем данные для Unifloc
        fig.add_trace(go.Scatter(x=liquid_rate_unifloc, y=pressure_unifloc, mode='lines',
                                name=f'Unifloc (GOR = {gor_value})', line=dict(color=color)))

    # Настройки графика
    fig.update_layout(
        title='Dependence of Discharge Pressure on Liquid Flow Rate for Different GOR values',
        xaxis_title='Liquid Flow Rate (sm3/day)',
        yaxis_title='Pressure (atma)',
        legend_title='Source'
    )

    # Показываем график
    fig.show()
else:
    print("Результаты расчета отсутствуют.")